In [5]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
import pinecone
from pinecone import Pinecone, ServerlessSpec

In [3]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [4]:
import json
data=json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Computer Science',
  'stars': 4,
  'review': 'Dr. Smith is very knowledgeable and engaging. His lectures are well-organized, but he can be a bit strict on deadlines.'},
 {'professor': 'Prof. Emily Davis',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'Prof. Davis is amazing! She explains concepts clearly and is always available for extra help.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Dr. Brown is decent, but his lectures can be a bit dry. You have to put in extra effort to do well.'},
 {'professor': 'Prof. Sarah Johnson',
  'subject': 'History',
  'stars': 5,
  'review': 'Prof. Johnson is fantastic! Her classes are interactive, and she really cares about her students.'},
 {'professor': 'Dr. William Lee',
  'subject': 'Biology',
  'stars': 2,
  'review': "Dr. Lee's classes are difficult to follow, and he is not very approachable during office hours."},
 {'professor': 'Prof. Jessica Ta

In [7]:
processed_data=[]
client=OpenAI()

for review in data['reviews']:
    response= client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",

    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id":review["professor"],
        "metadata":{
            "review":review["review"],
            "subject":review["subject"],
            "stars":review["stars"]

        }
    })

In [8]:
processed_data[0]

{'values': [-0.003023001,
  0.033698577,
  0.0014309556,
  -0.011769825,
  0.017288001,
  0.011598452,
  0.016876709,
  -0.020139629,
  0.01783639,
  -0.0037599,
  0.0023649326,
  -0.009130697,
  -0.0044625252,
  0.014628308,
  0.02344368,
  -0.020578342,
  -0.04082765,
  -0.044337347,
  -0.01675332,
  0.03635827,
  0.023731586,
  -0.024718687,
  0.048230916,
  -0.016383156,
  -0.053961594,
  -0.003449717,
  0.02567837,
  0.043213148,
  0.050040603,
  0.03342438,
  0.05042448,
  -0.016671062,
  0.011982325,
  0.0017685584,
  -0.036659885,
  0.05319385,
  0.0019862007,
  0.032684054,
  0.006899434,
  0.008746823,
  0.0006833456,
  -0.0020376123,
  -0.048697047,
  -0.013161364,
  0.031943727,
  -0.04751801,
  0.029832426,
  -0.037071176,
  0.028735645,
  0.03901796,
  -0.03745505,
  0.03177921,
  0.060980987,
  -0.021510605,
  -0.038387313,
  -0.0037941746,
  -0.004345992,
  0.050616413,
  0.030819528,
  -0.0030264284,
  0.079023026,
  -0.009343198,
  -0.0062790676,
  -0.034301806,
  -0.

In [9]:
index=pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [10]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}